In [6]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
# os.environ['JAVA_HOME']='/usr'
# os.environ['SPARK_HOME'] ='/usr/local/lib/python3.9/site-packages/pyspark/'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
# from pyspark.sql.functions import lpad, rpad

ModuleNotFoundError: No module named 'pandas'

In [2]:
conf = SparkConf().setAppName('ShowWithCollect')\
    .setMaster('yarn') \
    .set('spark.default.parallelism', '4')

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/19 19:48:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/19 19:48:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/19 19:48:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/07/19 19:48:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/07/19 19:48:52 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/07/19 19:48:52 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/07/19 19:48:53 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/07/19 19:48:53 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back 

app_id                                             application_1711527309433_0430


In [3]:
import random
import pyspark.sql.functions as F
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-365)
length_customers = 100

names = ['Bob', 'James', 'Marek', 'Johannes', 'Pierre', 'Jean', 'Thomas']
countries = ['bel', 'usa', 'fra', 'aus', 'ger', 'pol']

In [4]:
customers = list()
for i in range(length_customers):
    random_date = start + (end - start) * random.random()
    customer = (names[random.randint(0, len(names)-1)],         # name
                countries[random.randint(0, len(countries)-1)], # country
                random_date,                                    # last_order_date
                random.randint(-10000, 10000)               # balance
                )                                    
    customers.append(customer)

In [5]:
dfCust = spark.createDataFrame(data=customers, schema=["name","country","last_ord_dt", "bal"])
collected = dfCust.collect()
print(type(collected))

([' | '.join([i.name.ljust(12, ' '), 
              i.country, 
              str(i.last_ord_dt), 
              str(i.bal).rjust(10, ' ')]) for i in collected])

<class 'list'>


['Pierre       | aus | 2024-04-22 |      -5525',
 'Thomas       | usa | 2023-12-13 |      -1361',
 'Marek        | fra | 2024-03-22 |       7126',
 'Marek        | fra | 2023-12-25 |       9009',
 'James        | fra | 2023-08-06 |      -7107',
 'Johannes     | pol | 2023-12-06 |       9736',
 'Pierre       | bel | 2023-11-13 |       3252',
 'Pierre       | bel | 2024-01-24 |       4165',
 'James        | pol | 2024-04-05 |       5470',
 'James        | pol | 2024-06-26 |       5945',
 'Pierre       | fra | 2024-07-18 |      -7351',
 'Bob          | bel | 2023-11-19 |      -5309',
 'Johannes     | aus | 2023-11-17 |      -3875',
 'James        | pol | 2023-10-10 |      -6251',
 'James        | fra | 2024-07-09 |       1641',
 'Bob          | usa | 2024-02-13 |        692',
 'Bob          | bel | 2023-10-23 |      -1735',
 'Thomas       | ger | 2024-02-13 |      -1841',
 'Bob          | usa | 2024-06-23 |      -4630',
 'Bob          | ger | 2024-03-14 |      -6421',
 'Johannes     | aus

In [ ]:
# df_customers = df_customers.withColumn("name", F.rpad(F.col("name"), 10, " "))\
#     .withColumn("last_ord_dt", F.date_format(F.col("last_ord_dt"),"yyyy-MM-dd"))\
#     .withColumn("bal_str", F.format_number(F.col("bal").cast("double"), 2))\
#     .withColumn("bal", F.lpad(F.col("bal_str"), 12, " "))\
#     .drop("bal_str")

In [20]:
df_customers = df_customers.withColumn("name", F.rpad(F.col("name"), 10, " "))\
    .withColumn("last_ord_dt", F.date_format(F.col("last_ord_dt"),"yyyy-MM-dd"))\
    .withColumn("bal_str", F.format_number(F.col("bal").cast("double"), 2))\
    .withColumn("bal_str", F.regexp_replace(F.regexp_replace(F.regexp_replace(
            F.col("bal_str"), '\\.', '#'), ',', '\\.'), '#', ','))\
    .withColumn("bal", F.lpad(F.col("bal_str"), 14, " "))\
    .drop("bal_str")
    # .withColumn("bal_string", F.lpad(df_customers["bal_string"], 8, " "))\


    # .withColumn("bal", F.lpad(df_customers["bal"], 8, " "))\
# .withColumn("russian_format", F.regexp_replace(F.regexp_replace(F.regexp_replace(
#         F.col("bal_string"), '\\.', '#'), ',', ' '), '#', ','))\

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2024|    5|    5|
+----+-----+-----+



In [4]:
--

SyntaxError: invalid syntax (88891658.py, line 1)

In [25]:
spark.stop()